In [131]:
import os

import pandas as pd
import numpy as np

import scipy.stats as stats
from scipy.stats import ttest_ind

from datetime import datetime, timedelta

from tqdm.notebook import tqdm

In [2]:
URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-01/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

# Задание 4

Мы рассмотрели несколько вариантов добавления эффекта. Есть ли смысл думать о способе добавления эффекта при оценке вероятности ошибки II рода или все способы дают одинаковый результат? Результаты могут быть разными. Чтобы в этом убедиться, проведём численный эксперимент.

Допустим, в наш А/В-тест попадают все пользователи, совершавшие покупки до 28 марта.

 
Целевая метрика — средняя выручка с клиента за время эксперимента. Целевую метрику считаем на неделе с 21 по 28 марта. Уровень значимости — 0.05. Критерий — тест Стьюдента. Размер групп — 1000. Ожидаемый эффект — средняя выручка увеличится на 10%.

Нужно оценить вероятности ошибок II рода для трёх вариантов добавления эффекта:

1. Добавление константы ко всем значениям;

2. Умножение на константу всех значений;

3. Добавление константы к 2.5% значений.

In [5]:
df_sales = read_database('2022-04-01T12_df_sales.csv')
df_sales['date'] = pd.to_datetime(df_sales['date'])

In [117]:
df_users.shape

(98584,)

In [123]:
df_users = df_sales[df_sales['date'] < datetime(2022, 3,28)][['user_id']].drop_duplicates()
df_sales_exp = df_sales[(df_sales['date'] >= datetime(2022, 3, 21)) & (df_sales['date'] < datetime(2022, 3,28))].reset_index()
df_sales_by_user = df_sales_exp.groupby(['user_id'], as_index=False)['price'].sum()
df = df_users.merge(df_sales_by_user, how='left', on='user_id').fillna(0)

1. Добавление константы ко всем значениям;

In [124]:
alpha = 0.05
effect = 10 # в процентах
sample_size = 1000

users = df['user_id'].unique()

p_values = []

for i in range(1000):
    np.random.shuffle(users)
    group_a, group_b = users[:sample_size], users[sample_size:sample_size * 2]
    sales_a = df[df['user_id'].isin(group_a)]['price']
    sales_b = df[df['user_id'].isin(group_b)]['price']
    sales_b += sales_b.mean() * (effect / 100)
    _, p_val = ttest_ind(sales_a, sales_b)
    p_values.append(p_val)

In [125]:
print(f'Оценка вероятности ошибка 2-го рода с добавлением константы = {1 - (np.array(p_values) < 0.05).mean()}')

Оценка вероятности ошибка 2-го рода с добавлением константы = 0.806


2. Умножение на константу всех значений;

In [126]:
alpha = 0.05
effect = 10 # в процентах
sample_size = 1000

users = df['user_id'].unique()

p_values = []

for i in range(1000):
    np.random.shuffle(users)
    group_a, group_b = users[:sample_size], users[sample_size:sample_size * 2]
    sales_a = df[df['user_id'].isin(group_a)]['price']
    sales_b = df[df['user_id'].isin(group_b)]['price']
    sales_b *= (1 + (effect / 100))
    _, p_val = ttest_ind(sales_a, sales_b)
    p_values.append(p_val)

In [127]:
print(f'Оценка вероятности ошибка 2-го рода с умножением на константу = {1 - (np.array(p_values) < 0.05).mean()}')

Оценка вероятности ошибка 2-го рода с умножением на константу = 0.833


3. Добавление константы к 2.5% значений.

In [128]:
alpha = 0.05
effect = 10 # в процентах
sample_size = 1000

users = df['user_id'].unique()

p_values = []

for i in range(1000):
    np.random.shuffle(users)
    group_a, group_b = users[:sample_size], users[sample_size:sample_size * 2]
    sales_a = df[df['user_id'].isin(group_a)]['price']
    sales_b = df[df['user_id'].isin(group_b)]['price']
    sales_b[:int(sample_size * 0.025)] += sales_b.mean() * (effect / 2.5) 
    _, p_val = ttest_ind(sales_a, sales_b)
    p_values.append(p_val)

/var/folders/1p/ybd4mcmj71j_w9gphd0xdrgc6vsdmk/T/ipykernel_789/20926853.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_b[:int(sample_size * 0.025)] += sales_b.mean() * (effect / 2.5)


In [129]:
print(f'Оценка вероятности ошибка 2-го рода с добавлением константы части наблюдений = {1 - (np.array(p_values) < 0.05).mean()}')

Оценка вероятности ошибка 2-го рода с добавлением константы части наблюдений = 0.8109999999999999


## Решение из курса

In [132]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
from scipy import stats
from tqdm.notebook import tqdm

URL_BASE = 'https://raw.githubusercontent.com/ab-courses/simulator-ab-datasets/main/2022-04-01/'

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

df_sales = read_database('2022-04-01T12_df_sales.csv')
df_sales['date'] = pd.to_datetime(df_sales['date'])

begin_date = datetime(2022, 3, 21)
end_date = datetime(2022, 3, 28)
df_users = df_sales[df_sales['date'] < end_date][['user_id']].drop_duplicates()
df_metrics = (
    df_sales
    [(df_sales['date'] >= begin_date) & (df_sales['date'] < end_date)]
    .groupby('user_id')[['price']].sum()
    .reset_index()  
)
df = pd.merge(df_users, df_metrics, on='user_id', how='left').fillna(0)

alpha = 0.05
sample_size = 1000
effect = 0.1

pvalues = {'one': [], 'two': [], 'three': []}
values = df['price'].values
mean_ = values.mean()

for _ in tqdm(range(30000)):
    # выбираем случайные группы
    a, b = np.random.choice(values, (2, sample_size,), False)
    # добавляем эффект тремя способами
    b_one = b + mean_ * effect
    b_two = b * (1 + effect)
    indexes = np.random.choice(np.arange(sample_size), int(sample_size * 0.025), False)
    add_value = effect * mean_ * sample_size / len(indexes)
    mask = np.zeros(sample_size)
    mask[indexes] += 1
    b_three = b + mask * add_value
    # считаем и сохраняем p-value
    for b_, key in ((b_one, 'one',), (b_two, 'two',), (b_three, 'three',),):
        pvalues[key].append(stats.ttest_ind(a, b_).pvalue)

# считаем точечные оценки вероятностей ошибки II рода
for key, v in pvalues.items():
    errors = (np.array(v) > alpha).astype(int)
    part_errors = np.mean(errors)
    print(f'{key}: part errors = {part_errors:0.4f}')

# проверим, что отличия статистически значимые
print(stats.ttest_ind(pvalues['one'], pvalues['three']).pvalue)
print(stats.ttest_ind(pvalues['two'], pvalues['three']).pvalue)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html